# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('./spam', 'spam'))
data = data.append(dataFrameFromDirectory('./ham', 'ham'))


Let's have a look at that DataFrame:

In [2]:
data.head()

,class,message
./spam/00249.5f45607c1bffe89f60ba1ec9f878039a,spam,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
./spam/00373.ebe8670ac56b04125c25100a36ab0510,spam,ATTENTION: This is a MUST for ALL Computer Use...
./spam/00214.1367039e50dc6b7adb0f2aa8aba83216,spam,This is a multi-part message in MIME format.\n...
./spam/00210.050ffd105bd4e006771ee63cabc59978,spam,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
./spam/00033.9babb58d9298daa2963d4f514193d7d6,spam,This is the bottom line. If you can GIVE AWAY...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [3]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
train_data, test_data = train_test_split(data, test_size=0.1)

counts = vectorizer.fit_transform(train_data['message'].values)
classifier = MultinomialNB()
targets = train_data['class'].values

classifier.fit(counts, targets)
len(targets)

2700

Let's try it out:

In [4]:
examples = test_data['message'].values
answers = test_data['class'].values
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)

match, non_spam, non_ham = 0,0,0
for (i, prediction) in enumerate(predictions):
    if answers[i] == prediction:
        match += 1
    elif prediction == 'spam':
        non_spam += 1
    else:
        non_ham += 1
        
print('Matches:', match)
print('Misclassifications: as spam', non_spam, ', as ham', non_ham)

Matches: 291
Misclassifications: as spam 0 , as ham 9


## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.